# Prototype 1

In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Expression, SolverFactory, NonNegativeReals, minimize

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(bounds=(0, 10), within=NonNegativeReals)
model.y = Var(bounds=(0, 10), within=NonNegativeReals)

# Define objectives
model.obj1 = Expression(expr=model.x + 2 * model.y)
model.obj2 = Expression(expr=2 * model.x + model.y)

# Define constraints
model.con1 = Constraint(expr=model.x + model.y >= 5)
model.con2 = Constraint(expr=model.x - model.y <= 3)

# Activate only one objective at a time
# model.dummy_obj = Objective(expr=0, sense=minimize)  # Dummy objective to activate obj1

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Display results
# model.display()

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

## Scalarize Method

In [2]:
from pyomo.environ import Suffix, RangeSet, Param

# Define a suffix to store the objective expressions and their weights
model.objectives = Suffix(direction=Suffix.IMPORT)
model.objectives.update([(model.obj1, 0.5), (model.obj2, 0.5)])

model.objectives_set = RangeSet(0,len(model.objectives)-1)

model.objective_weights = Param(model.objectives_set, within=NonNegativeReals, mutable=True, initialize=0.0)
 

# Assign weights to the objectives
for i, (obj, weight) in enumerate(model.objectives.items()):
    model.objective_weights[i] = weight

'''
for obj, weight in model.objectives.items():
    print(f"Objective {obj} with weight {weight}")
'''
    
# Set the objective to minimize the weighted sum of the objectives
# model.scalarize_objective = Objective(expr=sum(weight * obj for obj, weight in model.objectives.items()), sense=minimize)

# This allows for the weights to be adjusted dynamically but not the expressions
# Another option would be to overwrite the objective with a new one
model.scalarize_objective = Objective(expr=sum(model.objective_weights[i] * obj for i, (obj, weight) in enumerate(model.objectives.items())), sense=minimize)

# Solve the model again with the new objective
results = solver.solve(model, tee=True)


Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [3]:
model.pprint()

1 RangeSet Declarations
    objectives_set : Dimen=1, Size=2, Bounds=(0, 1)
        Key  : Finite : Members
        None :   True :   [0:1]

1 Param Declarations
    objective_weights : Size=2, Index=objectives_set, Domain=NonNegativeReals, Default=None, Mutable=True
        Key : Value
          0 :   0.5
          1 :   0.5

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 2.0189514466411977 :    10 : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 2.981048505028896 :    10 : False : False : NonNegativeReals

2 Expression Declarations
    obj1 : Size=1, Index=None
        Key  : Expression
        None : x + 2*y
    obj2 : Size=1, Index=None
        Key  : Expression
        None : 2*x + y

1 Objective Declarations
    scalarize_objective : Size=1, Index=None, Active=True
        Key

## Determine Utopia Points

In [4]:
# Does updating the suffix automatically update the objective?
model.objectives.update([(model.obj1, 1), (model.obj2, 0.0)])
# No. Need to rebuild the objective.

# Rebuild the objective with the updated weights
model.scalarize_objective = Objective(expr=sum(weight * obj for obj, weight in model.objectives.items()), sense=minimize)



(type=<class 'pyomo.core.base.objective.ScalarObjective'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.objective.ScalarObjective'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [5]:
results = solver.solve(model, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [6]:
model.pprint()

1 RangeSet Declarations
    objectives_set : Dimen=1, Size=2, Bounds=(0, 1)
        Key  : Finite : Members
        None :   True :   [0:1]

1 Param Declarations
    objective_weights : Size=2, Index=objectives_set, Domain=NonNegativeReals, Default=None, Mutable=True
        Key : Value
          0 :   0.5
          1 :   0.5

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 3.9999999883317914 :    10 : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 0.9999999633380688 :    10 : False : False : NonNegativeReals

2 Expression Declarations
    obj1 : Size=1, Index=None
        Key  : Expression
        None : x + 2*y
    obj2 : Size=1, Index=None
        Key  : Expression
        None : 2*x + y

1 Objective Declarations
    scalarize_objective : Size=1, Index=None, Active=True
        K